In [43]:
import pandas as pd
import numpy as np
import torch

In [44]:
labels_df = pd.read_csv('data\labels.csv', header=None, names=['frame', 'exercise'])

In [ ]:
# What exercises was performed? How was it perfomed?

In [ ]:
# Dataset from where how it was collected and whats the datainside mean?

In [45]:
# where -001 mean no exercise was performed within those frames
# whole dataset it
labels_df

,frame,exercise
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
8403,8403,-1
8404,8404,-1
8405,8405,-1
8406,8406,-1


In [46]:
landmarks_df = pd.read_csv('data\landmarks.csv')

In [47]:
# Dropping face keypoints as they are not necessary for movement

head_landmarks = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER',
    'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT'
]

landmarks_df = landmarks_df[~landmarks_df['landmark'].isin(head_landmarks)]

In [48]:
def process_landmarks_data(df):
    pivoted_df = df.pivot(index='frame_number', 
                         columns='landmark', 
                         values=['x', 'y', 'z'])
    
    pivoted_df.columns = [f'{landmark}_{coord}' 
                         for coord, landmark in pivoted_df.columns]
    
    return pivoted_df.reset_index()

In [49]:
processed_landmarks_df = process_landmarks_data(landmarks_df)

In [50]:
df = pd.merge(processed_landmarks_df, 
                    labels_df,
                    left_on='frame_number',
                    right_on='frame',
                    how='left')

if 'frame' in df.columns:
    df = df.drop('frame', axis=1)

In [51]:
df['exercise'] = pd.to_numeric(df['exercise'])

In [52]:
df.head(5)

,frame_number,LEFT_ANKLE_x,LEFT_ELBOW_x,LEFT_FOOT_INDEX_x,LEFT_HEEL_x,LEFT_HIP_x,LEFT_INDEX_x,LEFT_KNEE_x,LEFT_PINKY_x,LEFT_SHOULDER_x,...,RIGHT_FOOT_INDEX_z,RIGHT_HEEL_z,RIGHT_HIP_z,RIGHT_INDEX_z,RIGHT_KNEE_z,RIGHT_PINKY_z,RIGHT_SHOULDER_z,RIGHT_THUMB_z,RIGHT_WRIST_z,exercise
0,0,0.425988,0.494306,0.428707,0.422518,0.435416,0.485270,0.433883,0.491423,0.461974,...,0.007676,0.154726,-0.000382,-0.396696,-0.026579,-0.388740,0.042103,-0.346265,-0.336874,-1
1,1,0.425991,0.492015,0.429230,0.422464,0.435435,0.486416,0.433899,0.491631,0.462259,...,-0.007274,0.134753,-0.000387,-0.379396,-0.027667,-0.369601,0.052328,-0.329296,-0.318225,-1
2,2,0.426002,0.492051,0.429614,0.422390,0.435449,0.486873,0.433885,0.491749,0.462260,...,-0.024269,0.114152,-0.000389,-0.383162,-0.032323,-0.371824,0.058376,-0.330526,-0.319684,-1
3,3,0.426009,0.491493,0.429729,0.422354,0.435497,0.489670,0.433865,0.493546,0.462321,...,-0.018429,0.120855,-0.001726,-0.383996,-0.034625,-0.384327,0.055875,-0.340302,-0.333625,-1
4,4,0.426016,0.491555,0.429806,0.422302,0.435549,0.490590,0.433850,0.494380,0.462301,...,-0.020860,0.117655,-0.002175,-0.389861,-0.035089,-0.393997,0.054835,-0.349526,-0.344045,-1


In [54]:
# saving table for future dataloader 

df.to_csv('data\data.csv', index=False)